In [1]:
from __future__ import division
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import os
import matplotlib
import numpy as np
import random
from numpy import linalg as LA
from PIL import Image

In [2]:
# '''This is the command to read in the GIF. You need Videocapture which makes a cv2 Video Object.
# Add your GIF file name in the parameter of this function.
# '''
# #gif_kiss = cv2.VideoCapture('test.gif')


# def convert_gif_to_frames(gif):

#     # Initialize the frame number and create empty frame list
#     frame_num = 0
#     frame_list = []

#     # Loop until there are frames left
#     while True:
#         try:
#             # Try to read a frame. Okay is a BOOL if there are frames or not
#             okay, frame = gif.read()
#             # Append to empty frame list
#             frame_list.append(frame)
#             # Break if there are no other frames to read
#             if not okay:
#                 break
#             # Increment value of the frame number by 1
#             frame_num += 1
#         except KeyboardInterrupt:  # press ^C to quit
#             break

#     return frame_list


# def output_frames_as_pics(frame_list):

#     # Reduce the list of frames by half to make the list more managable
#     frame_list_reduce = frame_list[0::2]
#     # Get the path of the current working directory
#     path = os.getcwd()
#     # Set then name of your folder
#     '''Replace this name with what you want your folder name to be'''
#     folder_name = 'Picturebook_Pics_Kiss'
#     # If the folder does not exist, then make it
#     if not os.path.exists(path + '/' + folder_name):
#         os.makedirs(path + '/' + folder_name)

#     for frames_idx in range(len(frame_list_reduce)):
#         cv2.imwrite(os.path.join(path + '/' + folder_name, str(frames_idx+1) + '.png'), frame_list_reduce[frames_idx])

# pass

In [3]:
# cap = cv2.VideoCapture('test1.gif')
# a = convert_gif_to_frames(cap)
# cv2.imshow('Original image',a[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [4]:
# a = convert_gif_to_frames(cap)

In [5]:
# img = Image.open('test.gif') # image extension *.png,*.jpg
# new_width  = 50
# new_height = 50
# img = img.resize((new_width, new_height), Image.ANTIALIAS)
# img.save('resize_images/test1.gif') # format may what u want ,*.png,*jpg,*.gif

In [6]:
# cap = cv2.imread('Aberdeen/lisa1.jpg')
# gray = cv2.cvtColor(cap, cv2.COLOR_BGR2GRAY)
# cv2.imshow('Original image',cap)
# cv2.imshow('Gray image', gray)
 
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [7]:
# len(a[0][0])

In [8]:
# len(cap[0][0])

In [9]:
# len(gray[0])

In [10]:
# plt.imshow(gray,cmap='gray')
# plt.show()

In [11]:
# images = []

In [12]:
# images

In [13]:
# new_width  = 50
# new_height = 50
# for root, dirs, files in os.walk('./face_dataset'):
#     k = 0
#     for name in files:
#         #print(name)
#         cap = cv2.VideoCapture('./face_dataset/'+name)
#         img = Image.open('./face_dataset/'+name) # image extension *.png,*.jpg
#         img = img.resize((new_width, new_height), Image.ANTIALIAS)
#         img.save('resize_images/'+name) # format may what u want ,*.png,*jpg,*.gif

In [14]:
# new_width  = 50
# new_height = 50
# for root, dirs, files in os.walk('./Aberdeen'):
#     k = 0
#     for name in files:
#         #print(name)
#         #cap = cv2.VideoCapture('./face_dataset/'+name))
#         img = Image.open('./Aberdeen/'+name).convert('LA') # image extension *.png,*.jpg
#         img = img.resize((new_width, new_height), Image.ANTIALIAS)
#         img.save('aberdeen_resize_images/'+name) # format may what u want ,*.png,*jpg,*.gif

In [15]:
img_name = []
d = {}
key = ''
for root, dirs, files in os.walk('./aberdeen_resize_images'):
    if not (root == './aberdeen_resize_images'):
        key = root.split('\\')[1]
        d[key] = []
    for name in files:
        img_name.append(name)
#         d[key].append(name)
        cap = cv2.imread('./aberdeen_resize_images/'+key+'/'+name)
#         a = convert_gif_to_frames(cap)
#         print(a[0][:])
        d[key].append(cap)
#         plt.imshow(a[0])
#         plt.show()

In [16]:
# d

In [17]:
# len(images)

In [18]:
def l1pca_SBF_rank1_simplified(dfX,L):
    N = len(dfX.T) #Number of data points
    max_iter = 25 #Number of iteration    
    delta = [0] * N #array to store quadratic value
    obj_val = 0 #initialize the objective function's value ||X*b||_2^2
    
    for i in range(0,L): #loop through initializations
#         b = []
#         for j in range(0,N): #random initalization of the b vector
#             temp = 0
#             while temp == 0:
#                 temp = random.randint(-1,1)
#             b.append(temp)
#         b = np.array(b)
        b = np.random.choice([0, 1], size=(N,), p=[1./3, 2./3])* 2 - 1;
        dfb = pd.DataFrame(b)
        print('Initializtion : ',i)
        
        for j in range(0,max_iter): #loop of iterations for bit flipping
            
            for k in range(0,N): #loop over N bits in b
#                 print('Initialization: ',i,'Iteration: ',j,'Sample: ',k)
                dfbk = dfb.copy()
                dfXk = dfX.copy()
                dfbk.drop([k])
                dfXk.drop([k],axis=1)
#                 bk = np.delete(b,k,0) #remove the i-th bit
#                 Xk = np.delete(X,k,1) #remove the i-th data sample
                delta[k] = int(-4 * (dfX.loc[:,0]*int(dfb.iloc[0])).dot(dfX.dot(dfb))) #calculate the "increase of the objective function" after flipping the i-th bit 
            ID = sorted(range(len(delta)), key=lambda g: delta[g],reverse = True) #sort delta and find the bit that leads to the highest increase
            #delta.sort(reverse = True)
            val = np.sort(delta)[::-1]
            
            if val[0] > 0: #if the highest "increase" is positive
                b[ID[0]] = -b[ID[0]] #then flip the corresponding bit in b
            else:
#                 print(j)
                break #otherwise, SBF terminates
        tmp = LA.norm(dfX.dot(dfb)) #calculate the  objective function's value
        if tmp > obj_val: #if larger than the old obj function value
            obj_val = tmp
            bopt = b #then update the obj function value, and the optimal b
            l_best = i
    Qprop = dfX.dot(dfb)/LA.norm(dfX.dot(dfb)) #PCA X*bopt/norm(X*bopt)
    Bprop = bopt
    return Qprop,Bprop,l_best

In [19]:
# images = np.array(images)

In [20]:
# images[0][0][0][0]

In [21]:
# len(d['lisa'][0][0])

In [22]:
ddtrain = {}
ddtest = {}

In [23]:

for i in d:
    item = 3
    ddtrain[i] = []
    ddtest[i] = []
    for j in d[i]:
        X = []
        for k in j:
            for l in k:
                X.append(k[0][0])
        if item > 0:
            ddtest[i].append(X[:])
        else:
            ddtrain[i].append(X[:])
        item -= 1

In [24]:
for i in ddtrain:
    print(i,len(ddtest[i]))

lisa 3
louise 3
marie 3
michael 3
nick 3
pat 3
stephen 3
trevor 3


In [25]:
# df = pd.DataFrame.from_dict(dd, orient='index')

In [26]:
# item = 0
# X = []
# for i in images:
#     tmp = []
#     for j in i:
#         for k in j:
#             tmp.append(k[0])
#     X.append(tmp)
#     item += 1

In [27]:
# len(X[90])

In [28]:
# len(images[0][0])

In [29]:
# m = 99999999999999
# for i in X:
#     m = min(len(i),m)

In [30]:
# m

In [31]:
# for i in range(0,len(X)):
#     X[i] = X[i][0:m]

In [32]:
# X = np.array(X)

In [33]:
# random.randrange(-1, 1, 2)

In [34]:
# len(X.T)

In [35]:
# b = [-1]*100

In [36]:
# dfX = pd.DataFrame(X)

In [37]:
# dfX

In [38]:
# dfb = pd.DataFrame(b)

In [39]:
#dfX.T.dot(dfb)/LA.norm(dfX.T.dot(dfb))

In [40]:
# (dfX.T.loc[:,0]*int(dfb.iloc[0])).dot(dfX.T.dot(dfb))

In [41]:
# df = pd.DataFrame([[0, 1, -2, -1], [1, 1, 1, 1]])
# other = pd.DataFrame([[0, 1], [1, 2], [-1, -1], [2, 0]])
# df.dot(other)

In [42]:
# df = pd.DataFrame

In [43]:
# df.iloc[0] = 0

In [44]:
# other

In [45]:
# dfX = pd.DataFrame(ddtrain['lisa'])

In [46]:
# len(dfX.T)

In [47]:
# Qprop,Bprop,l_best = l1pca_SBF_rank1_simplified(dfX.T,10)

In [48]:
# Qprop

In [49]:
dfrL1 = pd.DataFrame()
for i in ddtrain:
    dfX = pd.DataFrame(ddtrain[i])
    Qprop,Bprop,l_best = l1pca_SBF_rank1_simplified(dfX.T,10)
    print(i,l_best)
    dfrL1[i] = Qprop[0].values

Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
lisa 6
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
louise 3
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
marie 4
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
michael 8
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
nick 6
Initializ

In [50]:
dfrL1

,lisa,louise,marie,michael,nick,pat,stephen,trevor
0,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
1,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
2,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
3,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
4,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
5,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
6,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
7,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
8,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785
9,0.031828,0.020899,0.020214,0.020874,0.021973,0.021786,0.019823,0.020785


In [51]:
# df = pd.DataFrame()

In [52]:
# df['x'] = Qprop[0].values

In [53]:
# df

In [54]:
# Qprop,Bprop,l_best = l1pca_SBF_rank1_simplified(dfX.T,10)

In [55]:
# Qprop

In [56]:
# l_best

In [57]:
# newX = X.T - Qprop.dot(Qprop.T).dot(X.T)

In [58]:
# newX